In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.avro import functions as AF
from shared.spark_config import create_spark_config

from libs.configuration import configure

env = configure()
conf = create_spark_config("M2_Processors.aircraft.raw")

{'DATASTORAGE_AZURE_ACCESSKEY': 'dYj3PCGRyYHYR9pRo2xPa9BQ7t/sUonACJ29wcPdpa+1IC70rMbZmpfpHJJWJtk7OeASL+eQDIWq+AStGg3CCA==',
 'DATASTORAGE_AZURE_ACCOUNTNAME': 'icebergpe6t5cug7x9f45p8',
 'KAFKA_BOOTSTRAP_SERVERS': 'localhost:9092',
 'KAFKA_SCHEMA_REGISTRY_URL': 'http://localhost:8081',
 'KAFKA_SESSION_TIMEOUT_MS': '45000',
 'KAFKA_TOPIC_RAW_AIRCRAFT': 'raw.aircraft',
 'KAFKA_TOPIC_RAW_AIRLINE': 'raw.airline',
 'KAFKA_TOPIC_RAW_AIRPORT': 'raw.airport',
 'KAFKA_TOPIC_RAW_FLIGHT_SIGNAL': 'raw.flight_signal',
 'KAFKA_TOPIC_T1_AIRCRAFT': 't1.aircraft',
 'KAFKA_TOPIC_T1_AIRLINE': 't1.airline',
 'KAFKA_TOPIC_T1_AIRPORT': 't1.airport',
 'KAFKA_TOPIC_T1_FLIGHT_SIGNAL': 't1.flight_signal',
 'KAFKA_TOPIC_T2_AIRCRAFT': 't2.aircraft',
 'KAFKA_TOPIC_T2_AIRLINE': 't2.airline',
 'KAFKA_TOPIC_T2_AIRPORT': 't2.airport',
 'KAFKA_TOPIC_T2_FLIGHT_SIGNAL': 't2.flight_signal',
 'POSTGRE_JDBC_URL': 'jdbc:postgresql://localhost:5432/iceberg-catalog',
 'POSTGRE_PASSWORD': 'grebeci',
 'POSTGRE_USER': 'iceberg',
 

In [2]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
df = spark.read.csv(
    f"abfss://sampledata@{env.DATASTORAGE_AZURE_ACCOUNTNAME}.dfs.core.windows.net/aircrafts.csv",
    header=True,
    quote='"',
    inferSchema=True,
).drop(
    "manufacturername",
    "linenumber",
    "icaoaircrafttype",
    "testreg",
    "status",
    "firstflightdate",
    "seatconfiguration",
    "notes",
    "categoryDescription",
)

In [6]:
# schema = T.StructType(
#     [
#         T.StructField("icao24", T.StringType()),
#         T.StructField("registration", T.StringType()),
#         T.StructField("manufacturer_icao", T.StringType()),
#         T.StructField("manufacturer_name", T.StringType()),
#         T.StructField("model", T.StringType()),
#         T.StructField("model_icao", T.StringType()),
#         T.StructField("x", T.StringType()),
#         T.StructField("y", T.StringType()),
#         T.StructField("classification", T.StringType()),
#         T.StructField("operator", T.StringType()),
#         T.StructField("operator_callsign", T.StringType()),
#         T.StructField("operator_icao", T.StringType()),
#         T.StructField("operator_iata", T.StringType()),
#         T.StructField("owner", T.StringType()),
#         T.StructField("description", T.StringType()),
#     ]
# )

df = spark.read.csv("../../assets/aircrafts.csv", header=True, quote='"', inferSchema=True).drop(
    "manufacturername",
    "linenumber",
    "icaoaircrafttype",
    "testreg",
    "status",
    "firstflightdate",
    "seatconfiguration",
    "notes",
    "categoryDescription",
)

In [70]:
df.describe().show()

+-------+--------+-----------------+----------------+--------------------+--------+-------------+--------------------+----------------+-----------------+------------------+--------------------+--------------------+
|summary|  icao24|     registration|manufacturericao|               model|typecode| serialnumber|            operator|operatorcallsign|     operatoricao|      operatoriata|               owner|             engines|
+-------+--------+-----------------+----------------+--------------------+--------+-------------+--------------------+----------------+-----------------+------------------+--------------------+--------------------+
|  count|  457176|           454120|          376929|              387279|  421921|       384376|               20812|           35211|            36372|              7054|              383284|              244474|
|   mean|Infinity|78818.88443804035|            NULL|            Infinity|   737.0|     Infinity|                NULL|            NULL|     

In [7]:
df = df.withColumn("created_ts", F.current_timestamp())

In [9]:
df.schema

StructType([StructField('icao24', StringType(), True), StructField('registration', StringType(), True), StructField('manufacturericao', StringType(), True), StructField('model', StringType(), True), StructField('typecode', StringType(), True), StructField('serialnumber', StringType(), True), StructField('operator', StringType(), True), StructField('operatorcallsign', StringType(), True), StructField('operatoricao', StringType(), True), StructField('operatoriata', StringType(), True), StructField('owner', StringType(), True), StructField('registered', DateType(), True), StructField('reguntil', DateType(), True), StructField('built', DateType(), True), StructField('engines', StringType(), True), StructField('modes', BooleanType(), True), StructField('adsb', BooleanType(), True), StructField('acars', BooleanType(), True), StructField('created_ts', TimestampType(), False)])

In [11]:
df.writeTo("dev.raw.aircrafts").append()